# Meilisearch

TODO: Meilisearch presentation (https://www.meilisearch.com/)

The latest version of Meilisearch v1.3 supports vector search as an experimental feature.

This notebook shows you how to use functionality related to the Meilisearch vector database.

## Setup

### Run Meilisearch

TODO: Revision
To run this notebook you need a running Meilisearch instance with Vector Search capabilities enable.

See the following documentation for how to run a Meilisearch instance:
https://www.meilisearch.com/docs/learn/getting_started/quick_start.

If you are looking for a hosted Meilisearch, take a look at this:
https://www.meilisearch.com/pricing.

See the [documentation](https://www.meilisearch.com/docs/learn/experimental/vector-search) to find out how to enable Experimental feature: Vector Store.

In [1]:
!pip install meilisearch


To learn more about Meilisearch Python, refer to the in-depth
Meilisearch Python documentation: https://meilisearch.github.io/meilisearch-python/.

### Provide OpenAI access key
We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

If you don't want to have to send your client or Meilisearch instance address, you can directly configure the [environment variables](https://www.meilisearch.com/docs/learn/configuration/instance_options#environment-variables).

In [ ]:
os.environ["MEILI_HTTP_ADDR"] = getpass.getpass("Meilisearch HTTP address and port:")

In [ ]:
os.environ["MEILI_MASTER_KEY"] = getpass.getpass("Meilisearch API Key:")

## Add texts with user-friendly interface

In [5]:
from langchain.vectorstores import Meilisearch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

embeddings = OpenAIEmbeddings()

In [6]:
with open("../../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

There are several ways to instantiate the connection with Meilisearch, you can either send your `client` or just the `url` of your instance. `api_key` is optional, send it only if your instance needs a key.

In [ ]:
docsearch = Meilisearch.from_texts(texts=texts, embedding=embeddings)

## Add documents with user-friendly interface

In [4]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../../../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
docsearch = Meilisearch.from_documents(documents=documents, embedding=embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

## Add documents by creating Vectorstore

In [ ]:
from langchain.vectorstores import Meilisearch
import meilisearch

client = meilisearch.Client(url="http://127.0.0.1:7700", api_key="***")
vectorstore = Meilisearch(
    embedding=embeddings, client=client, index_name="langchain_demo", text_key="text"
)
vectorstore.add_documents(documents)

## Similarity Search with score

In [ ]:
docs_and_scores = docsearch.similarity_search_with_score(query)
docs_and_scores[0]

## Similarity Search by vector

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = docsearch.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]